<a href="https://colab.research.google.com/github/thaisbeham/Applied_deep_learning/blob/main/Assigment2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM - Fake News detection

Fake - 0
Real - 1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import pandas as pd
#from utils import merge_datasets, smaller_set
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Dense,LSTM,Dropout,Flatten,BatchNormalization,Conv1D,GlobalMaxPooling1D,MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import  SGD
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

#from hyperas.distributions import uniform

from keras.utils.np_utils import to_categorical
from keras import regularizers
import string
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



### Functions

In [5]:
def read_news(path, column_text_name):
  news = pd.read_csv(path)
  news[column_text_name] = news[column_text_name].astype(str)
  if column_text_name != "text":
    news.rename(columns = {column_text_name : "text"}, inplace = True)

  return news

def merge_split_datasets(real, fake, n_testsize = 0.15):
  # take 13000 rows from real data
  #real = real.sample(n = 13000, random_state= 2, axis = 0)

  # add column label
  real['label'] = 1
  fake['label'] = 0

  real = real[['text', 'label']]
  fake = fake[['text', 'label']]
  
  #merge real and fake
  # use only the sentence column on fake
  merged = pd.DataFrame(real.append(fake, ignore_index = True))
  merged = merged.dropna()
 
  merged = merged.sample(frac = 1, random_state= 1, ignore_index = True)#.reset_index()

  train, test = train_test_split(merged, test_size = n_testsize, random_state = 1)

  X_test = test['text']
  Y_test = test['label']

  #create validation set
  X = train['text']
  Y = train['label'].astype(int)    
  X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size=0.15, random_state=42)

  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

def predict_result(X_test, Y_test, tok):
  test_sequences = tok.texts_to_sequences(X_test)
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
  accr = model.evaluate(test_sequences_matrix,Y_test)
  print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
  predict_labels = model.predict(test_sequences_matrix) 

  # convert the range predicition into 0s or 1s
  Y_pred =[ 1 if i>0.5 else 0 for i in predict_labels  ] 

  print("Number Real news:",sum(Y_pred))
  print("Number Fake news:",len(Y_pred)-sum(Y_pred))
  print()
  print("Classification Report")
  print(classification_report(Y_test, Y_pred, target_names = ['class 0', 'class 1']))
  print()
  print("Confusion Matrix")
  matrix=confusion_matrix(Y_test,Y_pred,labels=[0,1])
  cm=pd.DataFrame(matrix,index=['class_0 pred','class_1 pred'],columns=['class_0 True','class_1 True'])
  print(cm)
  return

def trainer(X_train, Y_train, max_words, max_len, n_batchsize, n_epochs, model):
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(X_train)
  sequences = tok.texts_to_sequences(X_train)
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
  
  model.summary()
  model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
  model.fit(sequences_matrix,Y_train,batch_size=n_batchsize,epochs=n_epochs,
          validation_split=0.2)
  
  return model, tok



### Read datasets

In [6]:
#Read datasets
fake_news1 = read_news("/content/drive/MyDrive/dados_/fake.csv", "text")
fake_news2 = read_news("/content/drive/MyDrive/dados_/Fake_bisaillon.csv", "text")

real_news1 = read_news("/content/drive/MyDrive/dados_/df_2016.csv", "sentence")
real_news2 = read_news("/content/drive/MyDrive/dados_/True_bisaillon.csv", "text")

#shape of each dataset
print("fake_news1: ", np.shape(fake_news1), "\nfake_news2: ",  np.shape(fake_news2), "\nreal_news1: ",  np.shape(real_news1),"\nreal_news2: ", np.shape(real_news2)  )

real_news1_ = real_news1.sample(n = 15000, random_state= 2, axis = 0)


# separate the location of the news from the news text (e.g. "WASHINGTON (Reuters)")
real_news2[["loc", "text"]] = real_news2['text'].str.split('-',1, expand = True)
#real_news2.head()





fake_news1:  (12999, 20) 
fake_news2:  (23481, 4) 
real_news1:  (105606, 3) 
real_news2:  (21417, 4)


### **Train model with 2 datasets**

> Dataset fake.csv and df_2016.csv


In [7]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news1, fake_news1)

#### Parameters tunning

In [9]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 10

#### Train

In [10]:
model = RNN()

trained_model, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

#### Predict - Valid set

In [11]:
predict_result(X_valid, Y_valid, tok)

112/112 [==============================] - 1s 6ms/step - loss: 0.1543 - accuracy: 0.9630
Test set
  Loss: 0.154
  Accuracy: 0.963
112/112 [==============================] - 1s 5ms/step
Number Real news: 1964
Number Fake news: 1606

Classification Report
              precision    recall  f1-score   support

     class 0       0.96      0.96      0.96      1606
     class 1       0.97      0.97      0.97      1964

    accuracy                           0.96      3570
   macro avg       0.96      0.96      0.96      3570
weighted avg       0.96      0.96      0.96      3570


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          1540            66
class_1 pred            66          1898


#### Predict - Test set

In [12]:
predict_result(X_test, Y_test, tok)

132/132 [==============================] - 1s 6ms/step - loss: 0.1570 - accuracy: 0.9602
Test set
  Loss: 0.157
  Accuracy: 0.960
132/132 [==============================] - 0s 4ms/step
Number Real news: 2265
Number Fake news: 1935

Classification Report
              precision    recall  f1-score   support

     class 0       0.95      0.96      0.96      1920
     class 1       0.97      0.96      0.96      2280

    accuracy                           0.96      4200
   macro avg       0.96      0.96      0.96      4200
weighted avg       0.96      0.96      0.96      4200


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          1844            76
class_1 pred            91          2189


### **Train model with  4 datasets**

> Datasets: fake.csv, df_2016.csv, fake_Bisallion.csv and True_Bisallion.csv

In [13]:

real_news = pd.DataFrame(real_news1["text"].append(real_news2["text"], ignore_index = True))
fake_news = pd.DataFrame(fake_news1["text"].append(fake_news2["text"], ignore_index = True))
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = merge_split_datasets(real_news, fake_news)

#### Parameters tunning

In [14]:
max_words = 1000
max_len = 150
n_batchsize = 128
n_epochs = 10


#### Train model

In [15]:
model = RNN()

trained_model, tok = trainer(X_train, Y_train, max_words = max_words, max_len = max_len, n_batchsize = n_batchsize, n_epochs = n_epochs, model = model)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

#### Predict - Valid set

In [16]:
predict_result(X_valid, Y_valid, tok)

291/291 [==============================] - 2s 5ms/step - loss: 0.1416 - accuracy: 0.9580
Test set
  Loss: 0.142
  Accuracy: 0.958
291/291 [==============================] - 1s 4ms/step
Number Real news: 4889
Number Fake news: 4405

Classification Report
              precision    recall  f1-score   support

     class 0       0.98      0.94      0.96      4593
     class 1       0.94      0.98      0.96      4701

    accuracy                           0.96      9294
   macro avg       0.96      0.96      0.96      9294
weighted avg       0.96      0.96      0.96      9294


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          4304           289
class_1 pred           101          4600


#### Predict - test set

In [17]:
predict_result(X_test, Y_test, tok)

342/342 [==============================] - 2s 5ms/step - loss: 0.1330 - accuracy: 0.9570
Test set
  Loss: 0.133
  Accuracy: 0.957
342/342 [==============================] - 1s 4ms/step
Number Real news: 5707
Number Fake news: 5228

Classification Report
              precision    recall  f1-score   support

     class 0       0.97      0.94      0.96      5406
     class 1       0.94      0.97      0.96      5529

    accuracy                           0.96     10935
   macro avg       0.96      0.96      0.96     10935
weighted avg       0.96      0.96      0.96     10935


Confusion Matrix
              class_0 True  class_1 True
class_0 pred          5082           324
class_1 pred           146          5383
